# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [ ]:
# Libraries

import sys
# print(sys.path)
# Adding a path so that it finds the package for numpy
sys.path.append('/usr/local/lib/python3.7/site-packages')
import numpy as np

import pandas as pd
import re

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset admission predict from [here](https://drive.google.com/file/d/1y-8TOlbiSWUoMZ_Sg-WfOL2NySXP6i4e/view?usp=sharing]), place it in the data folder and load it

In [ ]:
# your code here

# Adm_df = pd.read_csv('../data/Admission_Predict.csv', sep=';' ,encoding='latin')
admissions = pd.read_csv('../data/Admission_Predict.csv')


Remove trailing spaces at the end of the column names if there are any.


In [ ]:
# your code here

admissions.columns
admissions = admissions.rename(columns={'Chance of Admit ':'Chance of Admit', 'LOR ': 'LOR'})


Let's evaluate the dataset by looking at the `head` function.

In [ ]:
# your code here

admissions.head()

Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [ ]:
# your code here

null_cols = admissions.isnull().sum()
null_cols

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [ ]:
# your code here

admissions.set_index(["Serial No."], inplace = True, 
                    append = False, drop = False) 
   
admissions.head()

Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [ ]:
# your code here
print(admissions.set_index(['GRE Score','CGPA']).index.is_unique)

Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [ ]:
# your code here

admissions.set_index(['GRE Score','CGPA'], inplace = True, 
                    append = False, drop = True) 
   
admissions.head()

In [ ]:
admissions.reset_index(inplace = True) 

admissions.head()

# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [ ]:
# your code here

My_adm = admissions.loc[(admissions['CGPA'] > 9) & (admissions['Research'] == 1)]
len(My_adm)


Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [ ]:
# your code here

My_adm = admissions.loc[(admissions['CGPA'] > 9) & (admissions['SOP'] < 3.5)]
len(My_adm)
#Paolo: and the mean chance?


We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [ ]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    
    # your code here
    
    return [(elem - np.mean(col))/np.std(col) for elem in col]
        

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [ ]:
# your code here

CGPA = list(admissions['CGPA'])
CGPA_std = standardize(CGPA)

GRE = list(admissions['GRE Score'])
GRE_std = standardize(GRE)

LOR = list(admissions['LOR'])
LOR_std = standardize(LOR)



In [ ]:
#Paolo: yes, but be careful you assigned column GCPA_std to LOR_std. I am changing it here
admissions['CGPA_std'] = CGPA_std
admissions['GRE_std'] = GRE_std
#admissions['LOR_std'] = CGPA_std
admissions['LOR_std'] = LOR_std #Paolo: I added this

In [ ]:
admissions.head()

We will generate the decision choice at random using the code below. Please run the cell.

In [ ]:
# Libraries
from random import choices

In [ ]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=admissions.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [ ]:
# your code here


admissions['decision_choice'] = decision_choice
admissions['deciding_column'] = admissions.lookup(admissions.index, admissions['decision_choice'])

admissions


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [ ]:
# your code here

admissions['decision'] = [1 if elem > 0.8 else 0 for elem in admissions['deciding_column']]
admissions.head()


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [ ]:
# your code here

Yes_adm = admissions.loc[(admissions['decision'] == 1)]
len(Yes_adm)

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [ ]:
# your code here



admissions.columns = [x.lower() for x in admissions.columns]
admissions.columns = admissions.columns.str.replace(" ", "_")
admissions.columns = admissions.columns.str.replace(".", "")

admissions.columns


In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [ ]:
# your code here

admissions['gre_score'] = np.where(admissions.university_rating >= 4, admissions.gre_score + 10, admissions.gre_score)

admissions.head()

In [ ]:
bins_label = ['Very_Low', 'Medium_Low', 'Medium_High', 'High']

admissions['adjusted_gre'] = pd.cut(admissions['gre_score'],4,labels=bins_label)
admissions.head()
#Paolo: very good! My suggestion  is not to modify the raw data of gre_score. Instead create a column with the adjusted numerical gre_score and then the adjusted_gre
# column has you have done. The difference is that you make clear the manipulation that you make on the raw data while leaving the raw data untouched.